# 📈 Dashboard com Gráficos Interativos

In [2]:
import sqlite3
import pandas as pd
import plotly.express as px
import gradio as gr
from sklearn.ensemble import IsolationForest

# 📥 Carregar e processar sessões a partir de todos os logs
def carregar_dados_sessoes():
    conn = sqlite3.connect("../database.db")
    query = """
        SELECT u.id AS usuario_id, u.nome, u.email, l.acao_usuario AS acao, l.data_hora
        FROM logs l
        JOIN usuarios u ON u.id = l.usuario_id
    """
    df = pd.read_sql_query(query, conn)
    conn.close()

    df['data_hora'] = pd.to_datetime(df['data_hora'])
    df['data'] = df['data_hora'].dt.strftime('%d/%m/%Y')

    # Agrupar sessões por usuário por dia
    df_sessoes = df.groupby(
        ['usuario_id', 'nome', 'email', 'data'],
        dropna=False
    ).agg(
        duracao_sessao=('data_hora', lambda x: (x.max() - x.min()).total_seconds() / 60 if len(x) > 1 else 0),
        acoes_realizadas=('acao', 'count')
    ).reset_index()

    return df_sessoes

# 🤖 Aplicar IA com Isolation Forest
def aplicar_classificacao_ia(df_sessoes):
    modelo = IsolationForest(contamination=0.2, random_state=42)

    X = df_sessoes[['duracao_sessao', 'acoes_realizadas']]
    modelo.fit(X)
    predicoes = modelo.predict(X)

    df_sessoes['classification'] = ['Anômala' if p == -1 else 'Normal' for p in predicoes]
    return df_sessoes

# 📊 Gráfico geral com base na classificação atual
def gerar_grafico_geral_classificacao(df_sessoes):
    df_class = df_sessoes['classification'].value_counts().reset_index()
    df_class.columns = ['Classificação', 'Total']

    cores = {
        'Normal': 'royalblue',
        'Anômala': 'crimson'
    }

    fig = px.bar(
        df_class,
        x='Classificação',
        y='Total',
        text='Total',
        color='Classificação',
        color_discrete_map=cores,
        title='📊 Classificação Geral das Sessões (IA)'
    )
    fig.update_traces(textposition='outside')
    return fig


# 🎯 Filtrar e gerar gráficos
def gerar_graficos(email="", nome="", usuario_id=""):
    df = carregar_dados_sessoes()
    df = aplicar_classificacao_ia(df)

    # Filtros
    if email:
        df = df[df['email'].str.contains(email, case=False, na=False)]
    if nome:
        df = df[df['nome'].str.contains(nome, case=False, na=False)]
    if usuario_id:
        try:
            usuario_id = int(usuario_id)
            df = df[df['usuario_id'] == usuario_id]
        except:
            return "⚠️ ID inválido.", None, None, None, None

    if df.empty:
        return "⚠️ Nenhum dado encontrado com os filtros aplicados.", None, None, None, None

    # 📈 Dispersão
    fig1 = px.scatter(
        df,
        x='duracao_sessao',
        y='acoes_realizadas',
        color='classification',
        hover_data=['usuario_id', 'nome', 'email', 'data'],
        title='📈 Sessões: Duração vs Ações Realizadas'
    )

    # 📊 Barras: sessões por usuário
    df_usuarios = df.groupby('nome').size().reset_index(name='quantidade_sessoes')
    fig2 = px.bar(df_usuarios, x='nome', y='quantidade_sessoes',
                  title='📊 Sessões por Usuário')

    # 🥧 Pizza
    df_class = df['classification'].value_counts().reset_index()
    df_class.columns = ['Classificação', 'Quantidade']
    fig3 = px.pie(df_class, names='Classificação', values='Quantidade',
                  title='🥧 Distribuição de Sessões (IA)')

    # 📊 Gráfico geral
    fig4 = gerar_grafico_geral_classificacao(df)

    return "✅ Dados carregados e classificados com sucesso!", fig1, fig2, fig3, fig4

# 🌐 Interface Gradio
with gr.Blocks(title="Dashboard de Sessões com IA") as app:
    gr.Markdown("## 📊 Dashboard de Sessões com Classificação por Inteligência Artificial")
    gr.Markdown("Filtre por ID, nome ou e-mail para explorar sessões normais e anômalas detectadas:")

    with gr.Row():
        id_input = gr.Textbox(label="🆔 ID do Usuário", placeholder="Ex: 1")
        nome_input = gr.Textbox(label="👤 Nome", placeholder="Ex: Ana")
        email_input = gr.Textbox(label="📩 E-mail", placeholder="Ex: ana@teste.com")

    botao = gr.Button("🔍 Gerar Gráficos")

    status = gr.Textbox(label="Status", interactive=False)
    graf1 = gr.Plot(label="📈 Duração vs Ações")
    graf2 = gr.Plot(label="📊 Sessões por Usuário")
    graf3 = gr.Plot(label="🥧 Sessões Classificadas")
    graf4 = gr.Plot(label="📊 Classificação Geral (IA)")

    botao.click(
        fn=gerar_graficos,
        inputs=[email_input, nome_input, id_input],
        outputs=[status, graf1, graf2, graf3, graf4]
    )

app.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
